In [1]:
import pandas as pd
import math
import numpy as np
import dill
import datetime
from datetime import timedelta
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import IntSlider
from ipywidgets import interact
from matplotlib import pylab as plt

In [2]:
#df_pan = pd.read_csv('medical.csv', low_memory=False)
#dill.dump(df_pan, open('df_pan.pkd', 'wb'))
df_prepan = pd.read_csv('Prepandemic_v2.csv')
df_pan = dill.load(open('df_pan.pkd', 'rb'))
df_pan_cases = pd.read_csv('casesAndDeaths.csv')
df_poverty = pd.read_csv('PovertyEstimates.csv')
df_pop = pd.read_csv('PopulationEstimates.csv')
df_policy = pd.read_csv('policy.csv')
df_county_cases = pd.read_csv('us-counties-covid.csv')

#Save it as csv; so streamlit won't have to process this all the time. Write it to csv

In [3]:
# Figure out later: 

#df_pan.to_csv('df_pan_test.csv', index = False)
#df_pan_test = pd.read_csv('df_pan_test.csv')
#df_pan_test.head()

/opt/conda/envs/data3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (123,124,125,126,127,128,129,130,131,132,133,134,870,871,872,873,874,875,876,877,878,879,880,881,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128,1129,1130,2575,2576,2577,2578,2579,2580,2581,2582,2583,2584,2585,2586,2587,2588,2589,2590,2591,2592,2593,2594,2595,2596,2597,2598,2599,2600,2601,2602,2603,2604,2605,2606,2607,2608,2609,2610,2611,2612,2613,2614,2615,2616,2617,2618,2619,4356,4357,4358,4359,4360,4361,4362,4363,4364,4365,4366,4367,11041,11042,11043,11044,11045,11046,11047,11048,11049,11050,11051,11052,11053,11054,11055,11056,11057,11058,11059,11060,11061,11062,11063,11064,11065,11066,11067,11068,11069,11070,11071,11072,11073,11074,11075,11076,11077,11078,11079,11080,11081,11082,11083,11084,11085) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,stfips,stname,ctyfips,ctyname,fips,ST_ID,FatalityRa,Confirmedb,DeathsbyPo,...,totalTestsViral_20200917,totalTestsViral_20200918,totalTestsViral_20200919,totalTestsViral_20200920,totalTestsViral_20200921,totalTestsViral_20200922,totalTestsViral_20200923,totalTestsViral_20200924,totalTestsViral_20200925,totalTestsViral_20200926
0,0,1,Alabama,1,Autauga County,1001,1,1.422880,3160.02,44.963220,...,1031253.0,1039010.0,1048422.0,1054017.0,1059517.0,1065553.0,1070903.0,1078180.0,1097595.0,1104932.0
1,1,1,Alabama,3,Baldwin County,1003,1,0.916422,2502.50,22.933465,...,1031253.0,1039010.0,1048422.0,1054017.0,1059517.0,1065553.0,1070903.0,1078180.0,1097595.0,1104932.0
2,2,1,Alabama,5,Barbour County,1005,1,0.801833,3508.70,28.133917,...,1031253.0,1039010.0,1048422.0,1054017.0,1059517.0,1065553.0,1070903.0,1078180.0,1097595.0,1104932.0
3,3,1,Alabama,7,Bibb County,1007,1,1.533742,2910.71,44.642857,...,1031253.0,1039010.0,1048422.0,1054017.0,1059517.0,1065553.0,1070903.0,1078180.0,1097595.0,1104932.0
4,4,1,Alabama,9,Blount County,1009,1,0.932836,2780.08,25.933610,...,1031253.0,1039010.0,1048422.0,1054017.0,1059517.0,1065553.0,1070903.0,1078180.0,1097595.0,1104932.0


In [3]:
#rename columns for df_policy
df_policy = df_policy.rename(columns={'Date closed K-12 public schools': 'Schools Closed', 
                         'Mandate face mask use by all individuals in public spaces': 'Mask Mandate', 
                         'Closed restaurants except take out':'Restaurants Closed'})
columns = ['stname','ctyname','fips','Stay at home/ shelter in place','Schools Closed', 'Mask Mandate', 'Restaurants Closed']
df_policy = df_policy[columns]
df_policy.head(3)



,stname,ctyname,fips,Stay at home/ shelter in place,Schools Closed,Mask Mandate,Restaurants Closed
0,Alabama,Autauga County,1001,2020/4/4,2020/3/20,2020/7/16,2020/3/19
1,Alabama,Baldwin County,1003,2020/4/4,2020/3/20,2020/7/16,2020/3/19
2,Alabama,Barbour County,1005,2020/4/4,2020/3/20,2020/7/16,2020/3/19


In [4]:
# Create table of positive test increases by date


df_first5col = df_pan[df_pan.columns[:6]]
df_posInc_only = df_pan[df_pan.columns[7804:8053]]
df_posInc = df_first5col.merge(df_posInc_only, left_index=True, right_index=True)
df_posInc.head(3)


,Unnamed: 0,stfips,stname,ctyfips,ctyname,fips,positiveIncrease_20200122,positiveIncrease_20200123,positiveIncrease_20200124,positiveIncrease_20200125,...,positiveIncrease_20200917,positiveIncrease_20200918,positiveIncrease_20200919,positiveIncrease_20200920,positiveIncrease_20200921,positiveIncrease_20200922,positiveIncrease_20200923,positiveIncrease_20200924,positiveIncrease_20200925,positiveIncrease_20200926
0,0,1,Alabama,1,Autauga County,1001,NaN,NaN,NaN,NaN,...,670.0,1106.0,1301.0,798.0,818.0,804.0,569.0,1053.0,2452.0,933.0
1,1,1,Alabama,3,Baldwin County,1003,NaN,NaN,NaN,NaN,...,670.0,1106.0,1301.0,798.0,818.0,804.0,569.0,1053.0,2452.0,933.0
2,2,1,Alabama,5,Barbour County,1005,NaN,NaN,NaN,NaN,...,670.0,1106.0,1301.0,798.0,818.0,804.0,569.0,1053.0,2452.0,933.0


In [5]:
datetime.datetime.strptime('2021/1/2', '%Y/%m/%d')

datetime.datetime(2021, 1, 2, 0, 0)

In [6]:
df_posInc_policy = df_policy.merge(df_posInc, on = ['stname','ctyname','fips'])
df_posInc_policy = df_posInc_policy.drop(columns=['Unnamed: 0','stfips','ctyfips'], axis=1)
#df_posInc_policy['Schools Closed'] = pd.to_datetime(df_posInc_policy['Schools Closed'])

def convert_to_dt(x, col_name):
    converted_col = pd.Series([ datetime.datetime.strptime(item, '%Y/%m/%d') for item in x[col_name] ])
    x[col_name] = converted_col

#convert_to_dt(df_posInc_policy, 'Schools Closed')

df_posInc_policy.head(3)

#Stopped here on 2/23/2021
#Next Steps: 
# Convert policy dates to datetime; there appears to be '0' values as dates. Find out how to bypass.
# - Transform positiveIncrease dates from columns into rows
# - group by county
# - Calculate 7-day rolling averages of infection rate before and after each intervention
# - Include cluster column into df

,stname,ctyname,fips,Stay at home/ shelter in place,Schools Closed,Mask Mandate,Restaurants Closed,positiveIncrease_20200122,positiveIncrease_20200123,positiveIncrease_20200124,...,positiveIncrease_20200917,positiveIncrease_20200918,positiveIncrease_20200919,positiveIncrease_20200920,positiveIncrease_20200921,positiveIncrease_20200922,positiveIncrease_20200923,positiveIncrease_20200924,positiveIncrease_20200925,positiveIncrease_20200926
0,Alabama,Autauga County,1001,2020/4/4,2020/3/20,2020/7/16,2020/3/19,NaN,NaN,NaN,...,670.0,1106.0,1301.0,798.0,818.0,804.0,569.0,1053.0,2452.0,933.0
1,Alabama,Baldwin County,1003,2020/4/4,2020/3/20,2020/7/16,2020/3/19,NaN,NaN,NaN,...,670.0,1106.0,1301.0,798.0,818.0,804.0,569.0,1053.0,2452.0,933.0
2,Alabama,Barbour County,1005,2020/4/4,2020/3/20,2020/7/16,2020/3/19,NaN,NaN,NaN,...,670.0,1106.0,1301.0,798.0,818.0,804.0,569.0,1053.0,2452.0,933.0


In [7]:
x_series = []
for item in df_posInc_policy['Schools Closed']:
    if item == '0': 
        pass
    else:
        x_series.append(datetime.datetime.strptime(item, '%Y/%m/%d'))
x_series


[datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 3, 20, 0, 0),
 datetime.datetime(2020, 

In [9]:
df_pan.head(3)

,Unnamed: 0,stfips,stname,ctyfips,ctyname,fips,ST_ID,FatalityRa,Confirmedb,DeathsbyPo,...,totalTestsViral_20200917,totalTestsViral_20200918,totalTestsViral_20200919,totalTestsViral_20200920,totalTestsViral_20200921,totalTestsViral_20200922,totalTestsViral_20200923,totalTestsViral_20200924,totalTestsViral_20200925,totalTestsViral_20200926
0,0,1,Alabama,1,Autauga County,1001,1,1.422880,3160.02,44.963220,...,1031253.0,1039010.0,1048422.0,1054017.0,1059517.0,1065553.0,1070903.0,1078180.0,1097595.0,1104932.0
1,1,1,Alabama,3,Baldwin County,1003,1,0.916422,2502.50,22.933465,...,1031253.0,1039010.0,1048422.0,1054017.0,1059517.0,1065553.0,1070903.0,1078180.0,1097595.0,1104932.0
2,2,1,Alabama,5,Barbour County,1005,1,0.801833,3508.70,28.133917,...,1031253.0,1039010.0,1048422.0,1054017.0,1059517.0,1065553.0,1070903.0,1078180.0,1097595.0,1104932.0


In [8]:
columns = (['FIPS','stname','ctyname_x', 'tot_2018', 'popdensity_2018'])
df_prepan = df_prepan[columns]
df_prepan.head(3)

,FIPS,stname,ctyname_x,tot_2018,popdensity_2018
0,1001,Alabama,Autauga County,55601,36.113876
1,1003,Alabama,Baldwin County,218022,52.949490
2,1005,Alabama,Barbour County,24881,10.854899


In [9]:
columns = (['FIPStxt','PCTPOVALL_2019'])
df_poverty = df_poverty[columns].rename(columns={'FIPStxt': 'FIPS', 'PCTPOVALL_2019': 'PCT_POV'})
df_poverty.head(3)

,FIPS,PCT_POV
0,0,12.3
1,1000,15.6
2,1001,12.1


In [10]:
df_prepan_pov = pd.merge(df_prepan, df_poverty, on='FIPS')
df_prepan_pov.head(3)

,FIPS,stname,ctyname_x,tot_2018,popdensity_2018,PCT_POV
0,1001,Alabama,Autauga County,55601,36.113876,12.1
1,1003,Alabama,Baldwin County,218022,52.949490,10.1
2,1005,Alabama,Barbour County,24881,10.854899,27.1


In [11]:
columns = (['fips','AgedPop','TotalPop'])
df_age = df_pan[columns].rename(columns={'fips': 'FIPS'})
df_age.head(3)

,FIPS,AgedPop,TotalPop
0,1001,8050,55200
1,1003,40665,208107
2,1005,4634,25782


In [12]:
df_age['PCT_AGED'] = df_age['AgedPop']/ df_age['TotalPop']
df_age = df_age[(['FIPS','PCT_AGED'])]

In [13]:
df_prepan_pov_age = pd.merge(df_prepan_pov, df_age, on='FIPS')
df_prepan_pov_age.head(3)

,FIPS,stname,ctyname_x,tot_2018,popdensity_2018,PCT_POV,PCT_AGED
0,1001,Alabama,Autauga County,55601,36.113876,12.1,0.145833
1,1003,Alabama,Baldwin County,218022,52.949490,10.1,0.195404
2,1005,Alabama,Barbour County,24881,10.854899,27.1,0.179738


In [14]:
# Calculate SMA
# datetime the needed dates, calculate SMA before and after
#or convert datetime date of intervention into string, and access the column
#'positiveIncrease_20200122'

# def intervention_effect(row, intervention): 
#         item = df_posInc_policy[intervention][row]
#         item_dt = datetime.datetime.strptime(item, '%Y/%m/%d')
#         item_str = datetime.datetime.strftime(item_dt, '%Y%m%d')
#         col_name = 'positiveIncrease_{}'.format(item_str)

#         col_index_of_event_date = df_posInc_policy.columns.get_loc(col_name)
#         avg_7dayPosInc = np.mean(df_posInc_policy.iloc[row,col_index_of_event_date+7:col_index_of_event_date + 14])
#         return avg_7dayPosInc



In [17]:
# for i in range(100): 
#     print(intervention_effect(i, 'Schools Closed'))

# new_col = [lambda row: intervention_effect(intervention) for row in df_posInc_policy]
# new_col

In [15]:
#intervention = 'Schools Closed'

#df_posInc_policy['posInc_post2Week_7DayAvg_{}'.format(intervention)] = [lambda x: intervention_effect(intervention) for x in df_posInc_policy]

#df_posInc_policy['posInc_post2Week_7DayAvg_{}'.format(intervention)] = df_posInc_policy.apply(intervention_effect(intervention), axis = 1) 

In [14]:
# Convert df_county_cases['date'] values to datetime
df_county_cases['date'] = df_county_cases['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

In [15]:
# 'Mask Mandate', 'Restaurants Closed'

intervention = 'Schools Closed'
row_idx = 0

row_county_fips = df_posInc_policy['fips'][row_idx]
row_event_date = df_posInc_policy[intervention][row_idx]
row_event_date = datetime.datetime.strptime(row_event_date, '%Y/%m/%d')
pre_window_start = row_event_date - timedelta(days = 7)
post_window_start = row_event_date + timedelta(days = 6)
post_window_end = row_event_date + timedelta(days = 14)


df_isolated_county_pre = df_county_cases[(df_county_cases['fips'] == row_county_fips) 
                                         & (df_county_cases['date'] >= pre_window_start)  
                                         & (df_county_cases['date'] < row_event_date)]
df_isolated_county_post = df_county_cases[(df_county_cases['fips'] == row_county_fips) 
                                         & (df_county_cases['date'] >= post_window_start)  
                                         & (df_county_cases['date'] < post_window_end)]

#print(df_isolated_county_pre[:10])

try: 
    totalCases_preDay1 = df_isolated_county_pre.iloc[0,4]
    totalCases_preDay7 = df_isolated_county_pre.iloc[6,4]
    total_county_pop = df_pan['POP_ESTIMA'][row]
    pre_rolling7DayAvg_increase = (totalCases_preDay7 - totalCases_preDay1) / (7*total_county_pop)
except: 
    pre_rolling7DayAvg_increase = None
    
totalCases_postDay1 = df_isolated_county_post.iloc[0,4]
totalCases_postDay7 = df_isolated_county_post.iloc[7,4]
total_county_pop = df_pan['POP_ESTIMA'][row_idx]
post_rolling7DayAvg_increase = (totalCases_postDay7 - totalCases_postDay1) / (7*total_county_pop)

print(pre_rolling7DayAvg_increase)
print(post_rolling7DayAvg_increase)


#pre_7DayRollingAvg = np.mean()


# item_str = datetime.datetime.strftime(item_dt, '%Y%m%d')
# col_name = 'positiveIncrease_{}'.format(item_str)

# df_posInc_policy[col_name]
# print(col_name)
# col_index_of_event_date = df_posInc_policy.columns.get_loc(col_name)
# col_index_of_event_date

# #Next: find 7-day rolling average, 2 weeks from event date. 
# print(df_posInc_policy.iloc[0,col_index_of_event_date+7:col_index_of_event_date + 14])
# avg_7dayPosInc = np.mean(df_posInc_policy.iloc[0,col_index_of_event_date+7:col_index_of_event_date + 14])
# print('Positive Increase (7-day Average): ')
# print(avg_7dayPosInc)


None
1.0277307448221641e-05


In [67]:
def intervention_effect(row_idx, intervention):
    try: 
        row_county_fips = df_posInc_policy['fips'][row_idx]
        row_event_date = df_posInc_policy[intervention][row_idx]
        row_event_date = datetime.datetime.strptime(row_event_date, '%Y/%m/%d')
        #print(row_event_date)
        pre_window_start = row_event_date - timedelta(days = 7)
        post_window_start = row_event_date + timedelta(days = 13) # 3 weeks after intervention
        post_window_end = row_event_date + timedelta(days = 21) # 3 weeks after intervention

        df_isolated_county_pre = df_county_cases[(df_county_cases['fips'] == row_county_fips) 
                                             & (df_county_cases['date'] >= pre_window_start)  
                                             & (df_county_cases['date'] < row_event_date)]
        df_isolated_county_post = df_county_cases[(df_county_cases['fips'] == row_county_fips) 
                                             & (df_county_cases['date'] >= post_window_start)  
                                             & (df_county_cases['date'] < post_window_end)]

        #print(df_isolated_county_pre[:10])

        try: 
            totalCases_preDay1 = df_isolated_county_pre.iloc[0,4]
            #print('totalCases_preDay1: ', totalCases_preDay1)
            totalCases_preDay7 = df_isolated_county_pre.iloc[6,4]
            #print('totalCases_preDay7: ', totalCases_preDay7)
            total_county_pop = df_pan['POP_ESTIMA'][row_idx]
            #print('total_county_pop: ', total_county_pop)
            pre_rolling7DayAvg_increase = (totalCases_preDay7 - totalCases_preDay1) / (7*total_county_pop)
            #print('Pre 7day increase: ', pre_rolling7DayAvg_increase)
        except: 
            pre_rolling7DayAvg_increase = None
            #print('Pre 7day increase: ', pre_rolling7DayAvg_increase)
        try:
            totalCases_postDay1 = df_isolated_county_post.iloc[0,4]
            totalCases_postDay7 = df_isolated_county_post.iloc[7,4]
            total_county_pop = df_pan['POP_ESTIMA'][row_idx]
            post_rolling7DayAvg_increase = (totalCases_postDay7 - totalCases_postDay1) / (7*total_county_pop)
            #print('Post 7day increase: ', post_rolling7DayAvg_increase)
        except: 
            post_rolling7DayAvg_increase = None

        try: 
            diff = pre_rolling7DayAvg_increase - post_rolling7DayAvg_increase
        except: 
            diff = None

        return diff
    except: 
        return None

In [16]:
df_posInc_policy.iloc[67]['Mask Mandate']

'2020/4/24'

In [70]:
def createPrePostDiffCol(intervention):
    diffs = []
    for i in range(len(df_pan)):
        diff = intervention_effect(i, intervention)
        if i % 500 == 0: 
            print(i, diff)
        diffs.append(diff)
    col_name = 'pre_post_diff_{}'.format(intervention)
    df = pd.DataFrame(diffs, columns = [col_name])
    return df

intervention = 'Mask Mandate'
col_name = 'pre_post_diff_{}'.format(intervention)
df_posInc_policy[col_name] = createPrePostDiffCol(intervention)


0 2.3123941758498688e-05
500 None
1000 -1.6291403243726994e-05
1500 None
2000 0.0008449637248331925
2500 None
3000 None


In [72]:
intervention = 'Stay at home/ shelter in place'
col_name = 'pre_post_diff_{}'.format(intervention)
df_posInc_policy[col_name] = createPrePostDiffCol(intervention)

0 -1.027730744822164e-05
500 None
1000 None
1500 2.3597149464344707e-05
2000 None
2500 -2.6883165761600085e-06
3000 None


In [73]:
intervention = 'Restaurants Closed'
col_name = 'pre_post_diff_{}'.format(intervention)
df_posInc_policy[col_name] = createPrePostDiffCol(intervention)

0 None
500 None
1000 None
1500 None
2000 None
2500 None
3000 None


In [74]:
df_posInc_policy.head()


,stname,ctyname,fips,Stay at home/ shelter in place,Schools Closed,Mask Mandate,Restaurants Closed,positiveIncrease_20200122,positiveIncrease_20200123,positiveIncrease_20200124,...,positiveIncrease_20200920,positiveIncrease_20200921,positiveIncrease_20200922,positiveIncrease_20200923,positiveIncrease_20200924,positiveIncrease_20200925,positiveIncrease_20200926,pre_post_diff_Mask Mandate,pre_post_diff_Stay at home/ shelter in place,pre_post_diff_Restaurants Closed
0,Alabama,Autauga County,1001,2020/4/4,2020/3/20,2020/7/16,2020/3/19,NaN,NaN,NaN,...,798.0,818.0,804.0,569.0,1053.0,2452.0,933.0,0.000023,-0.000010,NaN
1,Alabama,Baldwin County,1003,2020/4/4,2020/3/20,2020/7/16,2020/3/19,NaN,NaN,NaN,...,798.0,818.0,804.0,569.0,1053.0,2452.0,933.0,-0.000056,-0.000017,NaN
2,Alabama,Barbour County,1005,2020/4/4,2020/3/20,2020/7/16,2020/3/19,NaN,NaN,NaN,...,798.0,818.0,804.0,569.0,1053.0,2452.0,933.0,0.000224,NaN,NaN
3,Alabama,Bibb County,1007,2020/4/4,2020/3/20,2020/7/16,2020/3/19,NaN,NaN,NaN,...,798.0,818.0,804.0,569.0,1053.0,2452.0,933.0,-0.000108,NaN,NaN
4,Alabama,Blount County,1009,2020/4/4,2020/3/20,2020/7/16,2020/3/19,NaN,NaN,NaN,...,798.0,818.0,804.0,569.0,1053.0,2452.0,933.0,-0.000094,-0.000017,NaN


In [15]:
item = df_posInc_policy['Schools Closed'][0]
item_dt = datetime.datetime.strptime(item, '%Y/%m/%d')
item_str = datetime.datetime.strftime(item_dt, '%Y%m%d')
col_name = 'positiveIncrease_{}'.format(item_str)

df_posInc_policy[col_name]
print(col_name)
col_index_of_event_date = df_posInc_policy.columns.get_loc(col_name)
col_index_of_event_date

#Next: find 7-day rolling average, 2 weeks from event date. 
print(df_posInc_policy.iloc[0,col_index_of_event_date+7:col_index_of_event_date + 14])
avg_7dayPosInc = np.mean(df_posInc_policy.iloc[0,col_index_of_event_date+7:col_index_of_event_date + 14])
print('Positive Increase (7-day Average): ')
print(avg_7dayPosInc)

positiveIncrease_20200320
positiveIncrease_20200327     81
positiveIncrease_20200328    109
positiveIncrease_20200329    110
positiveIncrease_20200330     53
positiveIncrease_20200331    122
positiveIncrease_20200401     96
positiveIncrease_20200402    156
Name: 0, dtype: object
Positive Increase (7-day Average): 
103.85714285714286


In [51]:
df_prepan_pov_age.to_pickle("./df_prepan_pov_age.pkl")

## Checkpoint

In [12]:
df_prepan_pov_age = pd.read_pickle("./df_prepan_pov_age.pkl")

In [13]:
columns = (['popdensity_2018', 'PCT_POV', 'PCT_AGED'])
df_prepan_pov_age_features = df_prepan_pov_age[columns]

In [14]:
df_prepan_pov_age_features.head(3)

,popdensity_2018,PCT_POV,PCT_AGED
0,36.113876,12.1,0.145833
1,52.949490,10.1,0.195404
2,10.854899,27.1,0.179738


In [16]:
standard_scaler = StandardScaler()
df_prepan_pov_age_scaled = pd.DataFrame(standard_scaler.fit_transform(df_prepan_pov_age_features),
                                          columns=df_prepan_pov_age_features.columns)

In [17]:
df_prepan_pov_age_scaled.head(3)

,popdensity_2018,PCT_POV,PCT_AGED
0,-0.099611,-0.406158,0.049994
1,-0.075433,-0.751041,0.647704
2,-0.135886,2.180469,0.458803


In [57]:
# pca3D = PCA(n_components=3)
# df_pca_3D = pca3D.fit_transform(df_prepan_pov_age_scaled)

In [58]:
for n_clusters in range(2, 9):
    km = KMeans(n_clusters=n_clusters)
    km.fit(df_prepan_pov_age_scaled)
    
    # Predict the cluster for each data point
    preds = km.predict(df_prepan_pov_age_scaled)

    # Calculate the mean silhouette coefficient for the number of clusters chosen
    score = silhouette_score(df_prepan_pov_age_scaled, preds, metric='euclidean')
    print('For n_clusters = {}, the average silhouette_score is : {}'
          .format(n_clusters, score.round(5)))

For n_clusters = 2, the average silhouette_score is : 0.47422
For n_clusters = 3, the average silhouette_score is : 0.48254
For n_clusters = 4, the average silhouette_score is : 0.4553
For n_clusters = 5, the average silhouette_score is : 0.41181
For n_clusters = 6, the average silhouette_score is : 0.41906
For n_clusters = 7, the average silhouette_score is : 0.37378
For n_clusters = 8, the average silhouette_score is : 0.37637


In [59]:
#Determining the clusters

n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, n_init=10)
kmeans.fit(df_prepan_pov_age_scaled)

cols = kmeans.predict(df_prepan_pov_age_scaled)
colors = ['#ba2121ff', '#42a5f5ff', '#efa016ff']
centroids = kmeans.cluster_centers_

def plot_rotate(az, el):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')

    ax.view_init(elev=el, azim=az)
    
    x = df_prepan_pov_age_scaled['popdensity_2018']
    y = df_prepan_pov_age_scaled['PCT_POV']
    z = df_prepan_pov_age_scaled['PCT_AGED']
    ax.set_xlabel('Pop. Density (persons/km^2)')
    ax.set_ylabel('Percent in Poverty')
    ax.set_zlabel('Percent of Seniors (Age 65+)', rotation=90, labelpad=-2)

    ax.scatter(x, y, z, marker='o', s=20, color=[colors[col] for col in cols])

interact(plot_rotate, az=IntSlider(min=0, max=300, step=10, value=50, description='azimith'), 
                      el=IntSlider(min=0, max=90, step=10, value=20, description='elevation'));

interactive(children=(IntSlider(value=50, description='azimith', max=300, step=10), IntSlider(value=20, descri…

In [60]:
# Invert standard scaler
true_centers = standard_scaler.inverse_transform(centroids)

In [61]:
typical_counties = pd.DataFrame(true_centers, columns=df_prepan_pov_age_scaled.columns, 
                                 index=['typical county 1', 'typical county 2', 
                                        'typical county 3'])
typical_counties

,popdensity_2018,PCT_POV,PCT_AGED
typical county 1,104.046459,13.867149,0.181342
typical county 2,34.032614,16.406796,0.008772
typical county 3,18426.840000,19.333333,0.137029


In [ ]:
df_prepan_pov_age_scaled[kmeans.labels_ == 1].index.values  #Indexes of typical county 1 in df. 

In [63]:
cluster1 = df_prepan_pov_age_scaled[kmeans.labels_ == 0] # lowest pop dens, lowest senior pop
cluster2 = df_prepan_pov_age_scaled[kmeans.labels_ == 1] # higher senior pop
cluster3 = df_prepan_pov_age_scaled[kmeans.labels_ == 2] # higher pop dens

In [70]:
orig_index_val = df_prepan_pov_age.index.values
cluster1_index_val = df_prepan_pov_age_scaled[kmeans.labels_ == 0].index.values
cluster2_index_val = df_prepan_pov_age_scaled[kmeans.labels_ == 1].index.values
cluster3_index_val = df_prepan_pov_age_scaled[kmeans.labels_ == 2].index.values

for i in orig_index_val:
    if i in cluster1_index_val:
        df_prepan_pov_age['Cluster'] = '1'
    elif i in cluster2_index_val:
        df_prepan_pov_age['Cluster'] = '2' 
    if i in cluster3_index_val:
        df_prepan_pov_age['Cluster'] = '3'

df_prepan_pov_age[df_prepan_pov_age['Cluster'] == '1'].head()

,FIPS,stname,ctyname_x,tot_2018,popdensity_2018,PCT_POV,PCT_AGED,Cluster
0,1001,Alabama,Autauga County,55601,36.113876,12.1,0.145833,1
1,1003,Alabama,Baldwin County,218022,52.949490,10.1,0.195404,1
2,1005,Alabama,Barbour County,24881,10.854899,27.1,0.179738,1
3,1007,Alabama,Bibb County,22400,13.894338,20.3,0.162516,1
4,1009,Alabama,Blount County,57840,34.632576,16.3,0.177518,1
